In [1]:
#importing python libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [2]:
import requests
import pandas as pd

#Call FRED API
# Define your API key
api_key = '2a2e9be69431487407cdf6e64bb58c72'

# Define the endpoint and parameters
endpoint = 'https://api.stlouisfed.org/fred/series/observations'
params = {'series_id': 'CUSR0000SEEB', 'api_key': api_key, 'file_type': 'json', 'observation_start': '2013-01-01'}

# Make the request
response = requests.get(endpoint, params=params)

# Parse the response
data = response.json()

# Create a pandas DataFrame
Fred_CC = pd.DataFrame(data['observations'])

# drop the first 2 columns
Fred_CC.drop(Fred_CC.columns[:3], axis=1, inplace=True)

# Convert the object 'value' to numeric
Fred_CC['value'] = pd.to_numeric(Fred_CC['value'])


In [3]:
# Reshape the data into quarterly columns
Fred_CC = pd.DataFrame(Fred_CC.values.reshape(-1,4), columns=['Q1_Wage', 'Q2_CH', 'Q3_CH', 'Q4_CH'])

In [4]:
Fred_CC.head()

,Q1_Wage,Q2_CH,Q3_CH,Q4_CH
0,634.935,636.563,638.068,639.683
1,641.058,642.074,644.160,646.535
2,647.333,649.092,651.362,653.524
3,654.259,656.944,658.904,660.584
4,662.441,664.260,665.773,667.762


In [5]:
Fred_CC.shape

(30, 4)

In [6]:
# Import bls library and connect to datasource, return as pd dataframe
import bls
df = bls.get_series('LEU0252881500', 2013, 2022)
df = pd.DataFrame({'date':df.index, 'value':df.values})

In [8]:
# Set date col as index and check info 
df.set_index('date', inplace = True)

df.info()

In [11]:
# Add new row to reshape in preperation for concat with FRED data
new_row = {'value': 0}
df = df.append(new_row, ignore_index=True)
df = pd.DataFrame(df.values.reshape(-1, 4),columns=['Q1_Wage','Q2_Wage','Q3_Wage','Q4_Wage'])

In [12]:
#Concatenate previous dataframes
df1 = pd.concat([Fred_CC, df], axis=1, join='inner')

#Ensure dataframe created successfully, check lenth, datatypes, and nulls in the data
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Q1_Wage  10 non-null     float64
 1   Q2_CH    10 non-null     float64
 2   Q3_CH    10 non-null     float64
 3   Q4_CH    10 non-null     float64
 4   Q1_Wage  10 non-null     float64
 5   Q2_Wage  10 non-null     float64
 6   Q3_Wage  10 non-null     float64
 7   Q4_Wage  10 non-null     float64
dtypes: float64(8)
memory usage: 768.0 bytes


In [13]:
df1.tail()

,Q1_Wage,Q2_CH,Q3_CH,Q4_CH,Q1_Wage,Q2_Wage,Q3_Wage,Q4_Wage
5,668.789,670.744,672.722,674.253,881.0,876.0,887.0,900.0
6,677.508,678.628,681.693,684.516,905.0,908.0,919.0,936.0
7,686.271,688.241,690.594,691.881,957.0,1002.0,994.0,984.0
8,694.559,695.652,697.440,698.556,989.0,990.0,1001.0,1010.0
9,699.997,701.688,703.289,704.842,1037.0,1041.0,1070.0,0.0


In [14]:
#Check proportion of income allocated towards childcare, Q1 of each year
df1.apply(lambda x: x['Q1_Wage'] / x['Q1_Wage'], axis=1)

,Q1_Wage,Q1_Wage
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,1.0
6,1.0,1.0
7,1.0,1.0
8,1.0,1.0
9,1.0,1.0


In [15]:
# Use LR model to create projected value for Q4, 2022
X = df1[['Q1_Wage','Q2_Wage','Q3_Wage']]
y = df1['Q4_Wage']
# Create training data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24)
# Define model type (Linear Regression) and fit model on data 
lr = LinearRegression()
lr.fit(X_train, y_train);
# Print scores for training and test data
print(lr.score(X_train, y_train))

print(lr.score(X_test, y_test))
# Create predictions from LR model 
predictions = lr.predict(X)

print(predictions)
# Check residuals
residuals = y - predictions
residuals

0.9999764624485912
-0.72262917827897
[ 786.35465158  798.76136703  824.58050721  849.00994987  874.77053556
  900.59914831  935.78595264  983.90842336  988.52871412 1010.02176617]


0      -0.354652
1       0.238633
2       0.419493
3      -0.009950
4     -17.770536
5      -0.599148
6       0.214047
7       0.091577
8      21.471286
9   -1010.021766
Name: Q4_Wage, dtype: float64

In [24]:
# Replace Q4, 2022 data with model projection, ensure data has been replaced successfully
df1['Q4_Wage'].replace(0, 1070, inplace = True )
df1.tail()

/Users/jeremymanning/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Q1_Wage,Q2_CH,Q3_CH,Q4_CH,Q1_Wage,Q2_Wage,Q3_Wage,Q4_Wage,Q1_Wage/Q1_Wage,Q2_CH/Q2_Wage,Q3_CH/Q3_Wage,Q4_CH/Q4_Wage
5,668.789,670.744,672.722,674.253,881.0,876.0,887.0,900.0,1.0,0.765689,0.758424,0.749170
6,677.508,678.628,681.693,684.516,905.0,908.0,919.0,936.0,1.0,0.747388,0.741777,0.731321
7,686.271,688.241,690.594,691.881,957.0,1002.0,994.0,984.0,1.0,0.686867,0.694763,0.703131
8,694.559,695.652,697.440,698.556,989.0,990.0,1001.0,1010.0,1.0,0.702679,0.696743,0.691640
9,699.997,701.688,703.289,704.842,1037.0,1041.0,1070.0,0.0,1.0,0.674052,0.657279,inf


In [17]:
# Use LR model to create projected value for Q4_CH (Childcare costs, 2022)
# Define new X and y variables
X1 = df1[['Q1_Wage','Q2_CH','Q3_CH']]
y1 = df1['Q4_CH']

# Create training data
X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=24)
# Fit model
lr.fit(X_train, y_train);
# Score model on train and test data

print(lr.score(X_train, y_train))

print(lr.score(X_test, y_test))

# Create, print predictions 
predictions = lr.predict(X1)

predictions

0.9999852863057099
0.9951242241747867


array([639.72898554, 646.4388542 , 653.64619518, 660.49919427,
       666.90224954, 674.25591732, 684.50368588, 691.90316762,
       698.22132051, 703.11317538])

In [18]:
# Replace Q4_CH, 2022 with predicted value, ensure successful replace
df1['Q4_CH'] = df1['Q4_CH'].replace(612, 819)
print(df1)

   Q1_Wage    Q2_CH    Q3_CH    Q4_CH  Q1_Wage  Q2_Wage  Q3_Wage  Q4_Wage
0  634.935  636.563  638.068  639.683    773.0    776.0    771.0    786.0
1  641.058  642.074  644.160  646.535    796.0    780.0    790.0    799.0
2  647.333  649.092  651.362  653.524    808.0    801.0    803.0    825.0
3  654.259  656.944  658.904  660.584    830.0    824.0    827.0    849.0
4  662.441  664.260  665.773  667.762    865.0    859.0    859.0    857.0
5  668.789  670.744  672.722  674.253    881.0    876.0    887.0    900.0
6  677.508  678.628  681.693  684.516    905.0    908.0    919.0    936.0
7  686.271  688.241  690.594  691.881    957.0   1002.0    994.0    984.0
8  694.559  695.652  697.440  698.556    989.0    990.0   1001.0   1010.0
9  699.997  701.688  703.289  704.842   1037.0   1041.0   1070.0      0.0


combine yearly wage averages, compare childcare expenses by year as proportion of salary

plot CC expenses vs. Wages

In [19]:
# Create dictionary to map quarters from income and childcare
dict = {'Q1_Wage':'Q1_Wage', 'Q2_Wage':'Q2_CH', 'Q3_Wage':'Q3_CH', 'Q4_Wage':'Q4_CH'}

In [23]:
# Divide each income record with mapped childcare value, create new corresponding column
for i in dict:
    df1[dict[i]+"/"+i] = df1[dict[i]]/df1[i]
print(df1)

   Q1_Wage    Q2_CH    Q3_CH    Q4_CH  Q1_Wage  Q2_Wage  Q3_Wage  Q4_Wage  \
0  634.935  636.563  638.068  639.683    773.0    776.0    771.0    786.0   
1  641.058  642.074  644.160  646.535    796.0    780.0    790.0    799.0   
2  647.333  649.092  651.362  653.524    808.0    801.0    803.0    825.0   
3  654.259  656.944  658.904  660.584    830.0    824.0    827.0    849.0   
4  662.441  664.260  665.773  667.762    865.0    859.0    859.0    857.0   
5  668.789  670.744  672.722  674.253    881.0    876.0    887.0    900.0   
6  677.508  678.628  681.693  684.516    905.0    908.0    919.0    936.0   
7  686.271  688.241  690.594  691.881    957.0   1002.0    994.0    984.0   
8  694.559  695.652  697.440  698.556    989.0    990.0   1001.0   1010.0   
9  699.997  701.688  703.289  704.842   1037.0   1041.0   1070.0      0.0   

   Q1_Wage/Q1_Wage  Q2_CH/Q2_Wage  Q3_CH/Q3_Wage  Q4_CH/Q4_Wage  
0              1.0       0.820313       0.827585       0.813846  
1              1.0  

In [ ]:
# Check proportional wage to childacare cost per Quarter/year
a = df1[['Q1_CH/Q1_Wage', 'Q2_CH/Q2_Wage', 'Q3_CH/Q3_Wage', 'Q4_CH/Q4_Wage']]
a